# Theoretical Deep Learning: Foundations of Why Neural Networks Work

## Learning Objectives

By the end of this notebook, you will understand:

1. **Universal Approximation Theory**
   - Why neural networks can approximate any function
   - The role of depth vs width
   - Connection to Neural Tangent Kernels

2. **Loss Landscape Geometry**
   - Why high-dimensional optimization isn't as hard as it seems
   - Mode connectivity and loss surface structure
   - Why SGD finds good solutions

3. **Implicit Regularization**
   - The implicit bias of gradient descent
   - Double descent phenomenon
   - Why overparameterized networks generalize

**Note**: This notebook is theory-focused with visualizations and empirical demonstrations rather than traditional exercises.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
from typing import Callable, List, Tuple
from tqdm import tqdm

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 4)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

---

## Part 1: Universal Approximation Theory

### 1.1 The Universal Approximation Theorem

**Theorem (Cybenko, 1989; Hornik et al., 1989)**: 
A feedforward neural network with:
- A single hidden layer
- Finite number of neurons
- Non-polynomial activation function

can approximate any continuous function on a compact subset of $\mathbb{R}^n$ to arbitrary accuracy.

**Mathematical Statement**:
For any continuous function $f: [0,1]^n \rightarrow \mathbb{R}$ and any $\epsilon > 0$, there exists a neural network $N$ such that:

$$|N(x) - f(x)| < \epsilon \quad \forall x \in [0,1]^n$$

**Key Insights**:
1. This tells us neural networks *can* approximate functions (existence)
2. It doesn't tell us *how many* neurons we need (efficiency)
3. It doesn't tell us *how to find* the weights (optimization)
4. It doesn't address *generalization* (learning from samples)

### 1.2 Intuition: Building Functions from Bumps

The proof works by showing that sigmoid/ReLU neurons can create "bump" functions, and any continuous function can be approximated by summing many bumps.

In [ ]:
def create_bump_function(center: float, width: float, height: float, 
                        activation: str = 'sigmoid') -> Callable:
    """Create a bump function using neural network building blocks."""
    def bump(x):
        if activation == 'sigmoid':
            # Two sigmoids create a bump: σ(w(x-c+w)) - σ(w(x-c-w))
            w = 1.0 / width
            return height * (torch.sigmoid(w * (x - center + width)) - 
                           torch.sigmoid(w * (x - center - width)))
        else:  # ReLU
            # ReLU bump: ReLU(x-c+w) - 2*ReLU(x-c) + ReLU(x-c-w)
            return height * (F.relu(x - center + width) - 
                           2 * F.relu(x - center) + 
                           F.relu(x - center - width)) / width
    return bump

# Demonstrate approximating a function with bumps
x = torch.linspace(-1, 1, 1000)

# Target function: a wiggly function
target = torch.sin(5 * x) * torch.exp(-x**2)

# Approximate with increasing numbers of bumps
fig, axes = plt.subplots(1, 4, figsize=(16, 3))

for idx, n_bumps in enumerate([5, 10, 20, 50]):
    # Create bumps at regular intervals
    centers = torch.linspace(-1, 1, n_bumps)
    width = 2.0 / n_bumps
    
    # Approximate: fit heights to match target
    approximation = torch.zeros_like(x)
    for center in centers:
        bump = create_bump_function(center, width, 1.0, 'sigmoid')
        # Simple fitting: height = target value at center
        center_idx = (torch.abs(x - center)).argmin()
        height = target[center_idx]
        bump_with_height = create_bump_function(center, width, height, 'sigmoid')
        approximation += bump_with_height(x)
    
    axes[idx].plot(x, target, 'b-', label='Target', linewidth=2, alpha=0.7)
    axes[idx].plot(x, approximation, 'r--', label='Approximation', linewidth=1.5)
    axes[idx].set_title(f'{n_bumps} Bumps\nMSE: {F.mse_loss(approximation, target):.4f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key observation: More bumps (neurons) → better approximation")
print("But the number of neurons needed grows exponentially with dimension!")

### 1.3 Width vs Depth: The Expressivity Trade-off

**Width** (neurons per layer):
- Universal approximation with 1 hidden layer
- May need exponentially many neurons
- Suffers from curse of dimensionality

**Depth** (number of layers):
- Can represent functions more efficiently
- Hierarchical feature composition
- Some functions require exponentially fewer parameters with depth

**Example**: Representing a Boolean function with $n$ inputs:
- Shallow network: $O(2^n)$ neurons needed
- Deep network: $O(n)$ neurons suffice

Let's demonstrate this with a compositional function:

In [ ]:
class WideNetwork(nn.Module):
    """Single hidden layer with many neurons."""
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class DeepNetwork(nn.Module):
    """Multiple hidden layers with fewer neurons each."""
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int, output_dim: int):
        super().__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

# Create a compositional function: f(x) = cos(sin(relu(x)))
# This naturally has a hierarchical structure
def compositional_function(x):
    return torch.cos(torch.sin(F.relu(x.sum(dim=1, keepdim=True))))

# Generate data
n_samples = 5000
input_dim = 10
X = torch.randn(n_samples, input_dim)
y = compositional_function(X)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

# Compare wide vs deep with same parameter budget
param_budget = 5000  # Approximately

# Wide network: 1 hidden layer
# params ≈ input_dim * hidden + hidden * output = 10 * h + h * 1
wide_hidden = param_budget // (input_dim + 1)  # ≈ 454
wide_net = WideNetwork(input_dim, wide_hidden, 1).to(device)

# Deep network: 5 hidden layers
# params ≈ input_dim * h + 4 * h * h + h * 1
# Solve: 10h + 4h² + h ≈ 5000 → h ≈ 31
deep_hidden = 31
deep_net = DeepNetwork(input_dim, deep_hidden, 5, 1).to(device)

print(f"Wide Network: {sum(p.numel() for p in wide_net.parameters()):,} parameters")
print(f"Deep Network: {sum(p.numel() for p in deep_net.parameters()):,} parameters")

# Train both networks
def train_network(model, loader, epochs=50):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = F.mse_loss(pred, y_batch)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        losses.append(epoch_loss / len(loader))
    
    return losses

print("\nTraining wide network...")
wide_losses = train_network(wide_net, loader)

print("Training deep network...")
deep_losses = train_network(deep_net, loader)

# Plot results
plt.figure(figsize=(10, 4))
plt.plot(wide_losses, label='Wide (1 layer, 454 neurons)', linewidth=2)
plt.plot(deep_losses, label='Deep (5 layers, 31 neurons each)', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Training Loss (MSE)')
plt.title('Width vs Depth: Same Parameter Budget')
plt.legend()
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nFinal losses:")
print(f"Wide network: {wide_losses[-1]:.6f}")
print(f"Deep network: {deep_losses[-1]:.6f}")
print(f"\nDeep network achieves {wide_losses[-1]/deep_losses[-1]:.2f}x lower loss!")

### 1.4 Neural Tangent Kernel (NTK) Perspective

**Key Idea**: In the limit of infinite width, neural network training behaves like kernel regression!

**The Neural Tangent Kernel**:
$$K_{\text{NTK}}(x, x') = \mathbb{E}_{\theta \sim \text{init}}\left[\left\langle \frac{\partial f(x; \theta)}{\partial \theta}, \frac{\partial f(x'; \theta)}{\partial \theta} \right\rangle\right]$$

**Why it matters**:
1. Explains why gradient descent finds global minima
2. Connects to classical kernel methods
3. In infinite width limit, the kernel is *constant* during training

**Training dynamics** become:
$$\frac{df(x; \theta(t))}{dt} = -\eta \sum_{i} K_{\text{NTK}}(x, x_i)(f(x_i; \theta(t)) - y_i)$$

This is a *linear* ODE in function space!

In [ ]:
def compute_ntk(model: nn.Module, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
    """Compute the Neural Tangent Kernel empirically.
    
    NTK(x1, x2) = <∂f(x1)/∂θ, ∂f(x2)/∂θ>
    """
    n1, n2 = x1.shape[0], x2.shape[0]
    kernel = torch.zeros(n1, n2)
    
    for i in range(n1):
        # Compute gradient of f(x1[i]) w.r.t. parameters
        model.zero_grad()
        out1 = model(x1[i:i+1])
        grad1 = torch.autograd.grad(out1, model.parameters(), 
                                    create_graph=False, retain_graph=False)
        grad1_flat = torch.cat([g.flatten() for g in grad1])
        
        for j in range(n2):
            # Compute gradient of f(x2[j]) w.r.t. parameters
            model.zero_grad()
            out2 = model(x2[j:j+1])
            grad2 = torch.autograd.grad(out2, model.parameters(), 
                                       create_graph=False, retain_graph=False)
            grad2_flat = torch.cat([g.flatten() for g in grad2])
            
            # Inner product of gradients
            kernel[i, j] = torch.dot(grad1_flat, grad2_flat).item()
    
    return kernel

# Demonstrate NTK behavior with different widths
input_dim = 5
n_points = 20
X_test = torch.randn(n_points, input_dim)

widths = [10, 50, 200, 1000]
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, width in enumerate(widths):
    # Create network
    model = nn.Sequential(
        nn.Linear(input_dim, width),
        nn.ReLU(),
        nn.Linear(width, 1)
    )
    
    # Initialize with scaled initialization (important for NTK regime)
    with torch.no_grad():
        for layer in model:
            if isinstance(layer, nn.Linear):
                layer.weight.data *= 1.0 / np.sqrt(width)
    
    # Compute NTK at initialization
    with torch.no_grad():
        ntk_init = compute_ntk(model, X_test, X_test)
    
    # Train for a few steps
    y_dummy = torch.randn(n_points, 1)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    for _ in range(100):
        optimizer.zero_grad()
        loss = F.mse_loss(model(X_test), y_dummy)
        loss.backward()
        optimizer.step()
    
    # Compute NTK after training
    with torch.no_grad():
        ntk_trained = compute_ntk(model, X_test, X_test)
    
    # Plot NTKs
    im1 = axes[0, idx].imshow(ntk_init.numpy(), cmap='viridis')
    axes[0, idx].set_title(f'Width={width}\nNTK at Init')
    plt.colorbar(im1, ax=axes[0, idx])
    
    im2 = axes[1, idx].imshow(ntk_trained.numpy(), cmap='viridis')
    axes[1, idx].set_title('NTK after Training')
    plt.colorbar(im2, ax=axes[1, idx])
    
    # Compute relative change
    rel_change = torch.norm(ntk_trained - ntk_init) / torch.norm(ntk_init)
    print(f"Width {width}: Relative NTK change = {rel_change:.4f}")

plt.tight_layout()
plt.show()

print("\nKey observation: As width increases, NTK changes less during training.")
print("In the infinite width limit, NTK is constant → training becomes kernel regression!")

---

## Part 2: Loss Landscape Geometry

### 2.1 The Mystery of High-Dimensional Optimization

**The Paradox**: 
- Neural networks have millions of parameters
- Loss landscape is non-convex with many local minima
- Yet gradient descent reliably finds good solutions

**Why?**

1. **Most critical points are saddle points, not local minima**
   - In high dimensions, most critical points (∇L = 0) have both positive and negative curvature
   - True local minima are exponentially rare
   - Gradient descent can escape saddle points

2. **Local minima are all roughly equally good**
   - Empirically, different local minima achieve similar test loss
   - This is not true for all non-convex problems!

3. **Mode connectivity**
   - Different minima are connected by paths of nearly constant loss
   - The loss landscape has a "flat" basin structure

### 2.2 Visualizing Loss Landscapes

We can visualize 2D slices of the loss landscape using random directions.

In [ ]:
def get_random_direction(model: nn.Module, normalize: bool = True) -> List[torch.Tensor]:
    """Generate a random direction in parameter space."""
    direction = []
    for param in model.parameters():
        if normalize:
            # Filter-wise normalization (better for visualization)
            random_dir = torch.randn_like(param)
            random_dir = random_dir / (torch.norm(random_dir) + 1e-10)
            random_dir = random_dir * torch.norm(param)
        else:
            random_dir = torch.randn_like(param)
        direction.append(random_dir)
    return direction

def set_weights(model: nn.Module, base_weights: List[torch.Tensor], 
                direction1: List[torch.Tensor], direction2: List[torch.Tensor],
                alpha: float, beta: float) -> None:
    """Set model weights to: base + alpha * dir1 + beta * dir2."""
    for param, base, d1, d2 in zip(model.parameters(), base_weights, direction1, direction2):
        param.data = base + alpha * d1 + beta * d2

def evaluate_loss_landscape(model: nn.Module, loader: DataLoader,
                           base_weights: List[torch.Tensor],
                           direction1: List[torch.Tensor], 
                           direction2: List[torch.Tensor],
                           range_val: float = 1.0, 
                           resolution: int = 25) -> np.ndarray:
    """Evaluate loss on a 2D grid around base_weights."""
    alphas = np.linspace(-range_val, range_val, resolution)
    betas = np.linspace(-range_val, range_val, resolution)
    
    loss_surface = np.zeros((resolution, resolution))
    
    model.eval()
    with torch.no_grad():
        for i, alpha in enumerate(alphas):
            for j, beta in enumerate(betas):
                set_weights(model, base_weights, direction1, direction2, alpha, beta)
                
                total_loss = 0
                for X_batch, y_batch in loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    pred = model(X_batch)
                    loss = F.mse_loss(pred, y_batch)
                    total_loss += loss.item()
                
                loss_surface[i, j] = total_loss / len(loader)
    
    # Restore base weights
    set_weights(model, base_weights, direction1, direction2, 0, 0)
    
    return loss_surface, alphas, betas

# Create a simple dataset
X_data = torch.randn(1000, 5)
y_data = torch.sin(X_data.sum(dim=1, keepdim=True))
dataset = TensorDataset(X_data, y_data)
loader = DataLoader(dataset, batch_size=100)

# Create and train a small network
model = nn.Sequential(
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 1)
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print("Training model...")
for epoch in range(100):
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(model(X_batch), y_batch)
        loss.backward()
        optimizer.step()

# Save trained weights
trained_weights = [p.data.clone() for p in model.parameters()]

# Generate random directions
dir1 = get_random_direction(model)
dir2 = get_random_direction(model)

# Evaluate loss landscape around trained weights
print("Evaluating loss landscape (this may take a minute)...")
loss_surface, alphas, betas = evaluate_loss_landscape(
    model, loader, trained_weights, dir1, dir2, range_val=0.5, resolution=20
)

# Visualize
fig = plt.figure(figsize=(14, 5))

# 3D surface plot
ax1 = fig.add_subplot(121, projection='3d')
A, B = np.meshgrid(alphas, betas)
surf = ax1.plot_surface(A, B, loss_surface.T, cmap=cm.viridis, alpha=0.8)
ax1.set_xlabel('Direction 1')
ax1.set_ylabel('Direction 2')
ax1.set_zlabel('Loss')
ax1.set_title('Loss Landscape (3D)')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Contour plot
ax2 = fig.add_subplot(122)
contour = ax2.contourf(A, B, loss_surface.T, levels=20, cmap='viridis')
ax2.plot(0, 0, 'r*', markersize=15, label='Trained weights')
ax2.set_xlabel('Direction 1')
ax2.set_ylabel('Direction 2')
ax2.set_title('Loss Landscape (Contour)')
ax2.legend()
fig.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

print(f"\nMinimum loss in landscape: {loss_surface.min():.6f}")
print(f"Loss at trained weights (center): {loss_surface[len(alphas)//2, len(betas)//2]:.6f}")

### 2.3 Mode Connectivity

**Observation (Garipov et al., 2018)**: Different SGD runs find different minima, but these minima are connected by paths of nearly constant low loss.

**Linear mode connectivity**: For some pairs of minima θ₁ and θ₂:
$$L(\alpha \theta_1 + (1-\alpha) \theta_2) \approx L(\theta_1) \approx L(\theta_2) \quad \forall \alpha \in [0,1]$$

**Why it matters**:
- Suggests the loss landscape has wide, flat basins
- Different solutions found by SGD are in the same "mode"
- Explains why ensembling and learning rate decay work

In [ ]:
def train_independent_model(dataset: TensorDataset, architecture: nn.Module, 
                          epochs: int = 50) -> nn.Module:
    """Train a model from random initialization."""
    model = architecture.to(device)
    loader = DataLoader(dataset, batch_size=100, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            loss = F.mse_loss(model(X_batch), y_batch)
            loss.backward()
            optimizer.step()
    
    return model

def interpolate_models(model1: nn.Module, model2: nn.Module, 
                      alpha: float) -> nn.Module:
    """Create a model with weights: alpha * w1 + (1-alpha) * w2."""
    interpolated = type(model1)().to(device)
    
    for p_interp, p1, p2 in zip(interpolated.parameters(), 
                                 model1.parameters(), 
                                 model2.parameters()):
        p_interp.data = alpha * p1.data + (1 - alpha) * p2.data
    
    return interpolated

def evaluate_model(model: nn.Module, loader: DataLoader) -> float:
    """Evaluate model loss."""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred = model(X_batch)
            loss = F.mse_loss(pred, y_batch)
            total_loss += loss.item()
    return total_loss / len(loader)

# Create dataset
n_samples = 2000
X = torch.randn(n_samples, 10)
y = torch.sin(X[:, :3].sum(dim=1, keepdim=True)) + 0.1 * torch.randn(n_samples, 1)
dataset = TensorDataset(X, y)
eval_loader = DataLoader(dataset, batch_size=100)

# Define architecture
def create_model():
    return nn.Sequential(
        nn.Linear(10, 50),
        nn.ReLU(),
        nn.Linear(50, 50),
        nn.ReLU(),
        nn.Linear(50, 1)
    )

# Train two independent models
print("Training Model 1...")
model1 = train_independent_model(dataset, create_model(), epochs=100)

print("Training Model 2...")
model2 = train_independent_model(dataset, create_model(), epochs=100)

# Evaluate along the linear path between them
alphas = np.linspace(0, 1, 21)
losses = []

print("Evaluating mode connectivity...")
for alpha in alphas:
    interpolated = interpolate_models(model1, model2, alpha)
    loss = evaluate_model(interpolated, eval_loader)
    losses.append(loss)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(alphas, losses, 'o-', linewidth=2, markersize=6)
plt.axhline(y=losses[0], color='r', linestyle='--', alpha=0.5, label='Model 1 loss')
plt.axhline(y=losses[-1], color='b', linestyle='--', alpha=0.5, label='Model 2 loss')
plt.xlabel('Interpolation parameter α')
plt.ylabel('Loss')
plt.title('Linear Mode Connectivity\n(α=0: Model 1, α=1: Model 2)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Compute barrier height
barrier_height = max(losses) - min(losses[0], losses[-1])
print(f"\nModel 1 loss: {losses[0]:.6f}")
print(f"Model 2 loss: {losses[-1]:.6f}")
print(f"Maximum loss on path: {max(losses):.6f}")
print(f"Barrier height: {barrier_height:.6f}")

if barrier_height < 0.01:
    print("\n✓ Strong mode connectivity! Models are in the same basin.")
else:
    print("\n⚠ Weak mode connectivity. Models may be in different basins.")
    print("  (Try training longer or with different hyperparameters)")

### 2.4 Why SGD Finds Good Minima

**Key insights**:

1. **Stochasticity helps escape sharp minima**
   - Sharp minima → high curvature → gradient noise is amplified
   - Flat minima → low curvature → gradient noise has less effect
   - SGD naturally prefers flat minima

2. **Flat minima generalize better**
   - Sharp minima are sensitive to perturbations
   - Flat minima are robust → better test performance

3. **Implicit regularization**
   - SGD implicitly regularizes toward simpler solutions
   - Related to the trajectory through parameter space

In [ ]:
def compute_sharpness(model: nn.Module, loader: DataLoader, epsilon: float = 0.01) -> float:
    """Compute sharpness: max loss increase when perturbing weights by epsilon.
    
    Sharpness measures the maximum eigenvalue of the Hessian (approximately).
    """
    original_loss = evaluate_model(model, loader)
    
    # Perturb in random directions and measure worst-case loss increase
    max_loss = original_loss
    
    for _ in range(10):  # Try 10 random directions
        # Save original weights
        original_weights = [p.data.clone() for p in model.parameters()]
        
        # Perturb
        with torch.no_grad():
            for p in model.parameters():
                perturbation = torch.randn_like(p) * epsilon
                p.data.add_(perturbation)
        
        # Evaluate
        perturbed_loss = evaluate_model(model, loader)
        max_loss = max(max_loss, perturbed_loss)
        
        # Restore
        with torch.no_grad():
            for p, orig in zip(model.parameters(), original_weights):
                p.data.copy_(orig)
    
    sharpness = (max_loss - original_loss) / epsilon
    return sharpness

# Train models with SGD vs full-batch GD
def train_with_optimizer(dataset: TensorDataset, batch_size: int, 
                        lr: float, epochs: int = 100) -> Tuple[nn.Module, List[float]]:
    """Train a model and return it with training losses."""
    model = nn.Sequential(
        nn.Linear(10, 30),
        nn.ReLU(),
        nn.Linear(30, 1)
    ).to(device)
    
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            loss = F.mse_loss(model(X_batch), y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        losses.append(epoch_loss / len(loader))
    
    return model, losses

# Create train and test sets
X_train = torch.randn(1000, 10)
y_train = torch.sin(X_train[:, :3].sum(dim=1, keepdim=True)) + 0.2 * torch.randn(1000, 1)
train_dataset = TensorDataset(X_train, y_train)

X_test = torch.randn(500, 10)
y_test = torch.sin(X_test[:, :3].sum(dim=1, keepdim=True)) + 0.2 * torch.randn(500, 1)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=100)

# Train with different batch sizes
print("Training with full-batch GD (batch_size=1000)...")
model_gd, losses_gd = train_with_optimizer(train_dataset, batch_size=1000, lr=0.1)

print("Training with mini-batch SGD (batch_size=32)...")
model_sgd, losses_sgd = train_with_optimizer(train_dataset, batch_size=32, lr=0.01)

# Evaluate test performance
test_loss_gd = evaluate_model(model_gd, test_loader)
test_loss_sgd = evaluate_model(model_sgd, test_loader)

# Compute sharpness
print("\nComputing sharpness (this may take a moment)...")
sharpness_gd = compute_sharpness(model_gd, test_loader)
sharpness_sgd = compute_sharpness(model_sgd, test_loader)

# Results
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Training curves
axes[0].plot(losses_gd, label='Full-batch GD', linewidth=2)
axes[0].plot(losses_sgd, label='Mini-batch SGD', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Training Curves')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Comparison
metrics = ['Test Loss', 'Sharpness']
gd_values = [test_loss_gd, sharpness_gd]
sgd_values = [test_loss_sgd, sharpness_sgd]

x = np.arange(len(metrics))
width = 0.35

axes[1].bar(x - width/2, gd_values, width, label='Full-batch GD', alpha=0.8)
axes[1].bar(x + width/2, sgd_values, width, label='Mini-batch SGD', alpha=0.8)
axes[1].set_ylabel('Value')
axes[1].set_title('Test Performance vs Sharpness')
axes[1].set_xticks(x)
axes[1].set_xticklabels(metrics)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nResults:")
print(f"Full-batch GD: Test loss = {test_loss_gd:.6f}, Sharpness = {sharpness_gd:.4f}")
print(f"Mini-batch SGD: Test loss = {test_loss_sgd:.6f}, Sharpness = {sharpness_sgd:.4f}")
print(f"\nSGD finds a {sharpness_gd/sharpness_sgd:.2f}x flatter minimum!")
print(f"Flatter minimum → {(test_loss_gd/test_loss_sgd - 1)*100:.1f}% better generalization")

---

## Part 3: Implicit Regularization and the Double Descent Phenomenon

### 3.1 Implicit Bias of Gradient Descent

**Key Question**: When there are multiple solutions that fit the training data perfectly, which one does gradient descent find?

**Answer**: Gradient descent has an *implicit bias* toward simpler solutions, even without explicit regularization!

**For linear models**: GD converges to the minimum ℓ₂-norm solution:
$$\theta_{\text{GD}} = \arg\min_{\theta: X\theta = y} \|\theta\|_2$$

**For neural networks**: The implicit bias is more complex, but roughly:
- Early stopping → implicit regularization
- SGD noise → prefers flat minima → better generalization
- Architecture → inductive bias (e.g., CNNs prefer local patterns)

### 3.2 The Double Descent Phenomenon

**Classical statistical learning**: 
- Underparameterized (# params < # samples) → underfitting
- Perfectly parameterized (# params ≈ # samples) → best performance
- Overparameterized (# params > # samples) → overfitting

**Modern deep learning**: This is wrong!

**Double descent curve**:
1. Classical regime: test error decreases as model size increases (underfitting → fitting)
2. Interpolation threshold: test error spikes when # params ≈ # samples
3. Modern regime: test error *decreases again* as we add more parameters!

**Why?**
- At interpolation threshold: many solutions fit training data, GD finds a bad one
- In overparameterized regime: implicit regularization kicks in, GD finds smoother solutions

In [ ]:
def create_model_with_width(width: int) -> nn.Module:
    """Create a 2-layer network with specified width."""
    return nn.Sequential(
        nn.Linear(10, width),
        nn.ReLU(),
        nn.Linear(width, 1)
    )

def train_to_convergence(model: nn.Module, train_loader: DataLoader, 
                        max_epochs: int = 500, patience: int = 50) -> Tuple[float, float]:
    """Train until convergence and return train and test losses."""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    best_train_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(max_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = F.mse_loss(pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        epoch_loss /= len(train_loader)
        
        if epoch_loss < best_train_loss - 1e-6:
            best_train_loss = epoch_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break
    
    # Evaluate
    model.eval()
    train_loss = evaluate_model(model, train_loader)
    test_loss = evaluate_model(model, test_loader)
    
    return train_loss, test_loss

# Create a small dataset to see double descent clearly
n_train = 100
n_test = 500
input_dim = 10

# Training data
X_train = torch.randn(n_train, input_dim)
# True function with noise
true_signal = torch.sin(X_train[:, 0]) + torch.cos(X_train[:, 1])
y_train = (true_signal + 0.3 * torch.randn(n_train)).unsqueeze(1)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test data (clean)
X_test = torch.randn(n_test, input_dim)
true_signal_test = torch.sin(X_test[:, 0]) + torch.cos(X_test[:, 1])
y_test = (true_signal_test + 0.3 * torch.randn(n_test)).unsqueeze(1)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=100)

# Sweep over model widths
widths = [1, 2, 3, 5, 8, 10, 15, 20, 30, 50, 80, 100, 150, 200]
train_losses = []
test_losses = []
n_params_list = []

print("Training models with different widths...")
for width in tqdm(widths):
    model = create_model_with_width(width).to(device)
    n_params = sum(p.numel() for p in model.parameters())
    n_params_list.append(n_params)
    
    train_loss, test_loss = train_to_convergence(model, train_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    print(f"Width {width:3d} ({n_params:5d} params): Train {train_loss:.4f}, Test {test_loss:.4f}")

# Plot double descent
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss vs width
axes[0].plot(widths, train_losses, 'o-', label='Train Loss', linewidth=2, markersize=6)
axes[0].plot(widths, test_losses, 's-', label='Test Loss', linewidth=2, markersize=6)
axes[0].axvline(x=n_train/(input_dim+1), color='r', linestyle='--', 
                alpha=0.5, label='Interpolation threshold')
axes[0].set_xlabel('Model Width (hidden layer size)')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Double Descent: Loss vs Model Width')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')

# Loss vs number of parameters
axes[1].plot(n_params_list, train_losses, 'o-', label='Train Loss', linewidth=2, markersize=6)
axes[1].plot(n_params_list, test_losses, 's-', label='Test Loss', linewidth=2, markersize=6)
axes[1].axvline(x=n_train, color='r', linestyle='--', 
                alpha=0.5, label=f'# training samples ({n_train})')
axes[1].set_xlabel('Number of Parameters')
axes[1].set_ylabel('Loss (MSE)')
axes[1].set_title('Double Descent: Loss vs Number of Parameters')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')

plt.tight_layout()
plt.show()

# Find the interpolation peak
peak_idx = np.argmax(test_losses)
print(f"\nDouble Descent Observations:")
print(f"Test loss peak at width={widths[peak_idx]} ({n_params_list[peak_idx]} parameters)")
print(f"Peak test loss: {test_losses[peak_idx]:.4f}")
print(f"Final test loss (width={widths[-1]}): {test_losses[-1]:.4f}")
print(f"Overparameterization helps! {(test_losses[peak_idx]/test_losses[-1]):.2f}x improvement.")

### 3.3 Why Overparameterization Helps: Lottery Ticket Hypothesis

**Lottery Ticket Hypothesis (Frankle & Carbin, 2019)**:
> A randomly-initialized, dense neural network contains a subnetwork that, when trained in isolation, can match the test accuracy of the original network.

**Implications**:
1. Large networks are easier to train because they contain many "winning lottery tickets"
2. Overparameterization helps optimization, not just representation
3. The right initialization matters: the winning ticket must have lucky initial weights

**Analogy**: 
- Small network: buying 1 lottery ticket (might not win)
- Large network: buying 1000 lottery tickets (likely to have a winner)

Let's demonstrate this with a simple pruning experiment:

In [ ]:
def prune_weights(model: nn.Module, prune_percent: float) -> nn.Module:
    """Prune smallest weights by magnitude."""
    # Collect all weights
    all_weights = []
    for param in model.parameters():
        if param.requires_grad:
            all_weights.extend(param.data.abs().flatten().tolist())
    
    # Find threshold
    threshold = np.percentile(all_weights, prune_percent)
    
    # Create mask
    masks = []
    for param in model.parameters():
        if param.requires_grad:
            mask = (param.data.abs() > threshold).float()
            masks.append(mask)
            param.data.mul_(mask)
    
    return masks

def apply_mask(model: nn.Module, masks: List[torch.Tensor]) -> None:
    """Apply saved masks to model (for retraining with same structure)."""
    mask_idx = 0
    for param in model.parameters():
        if param.requires_grad:
            param.data.mul_(masks[mask_idx])
            mask_idx += 1

# Create dataset
X_train = torch.randn(500, 10)
y_train = (torch.sin(X_train[:, 0]) + 0.2 * torch.randn(500)).unsqueeze(1)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test = torch.randn(200, 10)
y_test = (torch.sin(X_test[:, 0]) + 0.2 * torch.randn(200)).unsqueeze(1)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=100)

# 1. Train full network
print("1. Training full network...")
full_model = nn.Sequential(
    nn.Linear(10, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
).to(device)

# Save initial weights
init_weights = [p.data.clone() for p in full_model.parameters()]

# Train
optimizer = torch.optim.Adam(full_model.parameters(), lr=0.001)
for epoch in range(100):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(full_model(X_batch), y_batch)
        loss.backward()
        optimizer.step()

full_test_loss = evaluate_model(full_model, test_loader)
print(f"Full network test loss: {full_test_loss:.6f}")

# 2. Prune and continue training ("fine-tuning")
print("\n2. Pruning 80% of weights and fine-tuning...")
pruned_finetuned = nn.Sequential(
    nn.Linear(10, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
).to(device)

# Copy trained weights
with torch.no_grad():
    for p_pruned, p_full in zip(pruned_finetuned.parameters(), full_model.parameters()):
        p_pruned.data.copy_(p_full.data)

# Prune
masks = prune_weights(pruned_finetuned, 80)

# Fine-tune
optimizer = torch.optim.Adam(pruned_finetuned.parameters(), lr=0.001)
for epoch in range(50):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(pruned_finetuned(X_batch), y_batch)
        loss.backward()
        optimizer.step()
        apply_mask(pruned_finetuned, masks)  # Maintain sparsity

pruned_finetuned_loss = evaluate_model(pruned_finetuned, test_loader)
print(f"Pruned + fine-tuned test loss: {pruned_finetuned_loss:.6f}")

# 3. The Lottery Ticket: retrain from initialization with the same mask
print("\n3. Lottery Ticket: Retraining from initialization with same mask...")
lottery_ticket = nn.Sequential(
    nn.Linear(10, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
).to(device)

# Reset to initial weights
with torch.no_grad():
    for p_lottery, p_init in zip(lottery_ticket.parameters(), init_weights):
        p_lottery.data.copy_(p_init)

# Apply mask
apply_mask(lottery_ticket, masks)

# Retrain from scratch with mask
optimizer = torch.optim.Adam(lottery_ticket.parameters(), lr=0.001)
for epoch in range(100):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(lottery_ticket(X_batch), y_batch)
        loss.backward()
        optimizer.step()
        apply_mask(lottery_ticket, masks)  # Maintain sparsity

lottery_ticket_loss = evaluate_model(lottery_ticket, test_loader)
print(f"Lottery ticket test loss: {lottery_ticket_loss:.6f}")

# 4. Random pruning baseline
print("\n4. Baseline: Random 80% pruning from initialization...")
random_pruned = nn.Sequential(
    nn.Linear(10, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
).to(device)

# Reset to initial weights
with torch.no_grad():
    for p_random, p_init in zip(random_pruned.parameters(), init_weights):
        p_random.data.copy_(p_init)

# Random mask
random_masks = []
for param in random_pruned.parameters():
    if param.requires_grad:
        mask = (torch.rand_like(param) > 0.8).float()
        random_masks.append(mask)
        param.data.mul_(mask)

# Train
optimizer = torch.optim.Adam(random_pruned.parameters(), lr=0.001)
for epoch in range(100):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = F.mse_loss(random_pruned(X_batch), y_batch)
        loss.backward()
        optimizer.step()
        apply_mask(random_pruned, random_masks)

random_pruned_loss = evaluate_model(random_pruned, test_loader)
print(f"Random pruning test loss: {random_pruned_loss:.6f}")

# Visualize results
models = ['Full\nNetwork', 'Pruned +\nFine-tuned', 'Lottery\nTicket', 'Random\nPruning']
losses = [full_test_loss, pruned_finetuned_loss, lottery_ticket_loss, random_pruned_loss]
colors = ['blue', 'orange', 'green', 'red']

plt.figure(figsize=(10, 6))
bars = plt.bar(models, losses, color=colors, alpha=0.7)
plt.ylabel('Test Loss (MSE)')
plt.title('Lottery Ticket Hypothesis Demonstration\n(80% of weights pruned)')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (bar, loss) in enumerate(zip(bars, losses)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
             f'{loss:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n=== Key Findings ===")
print(f"1. Full network: {full_test_loss:.6f}")
print(f"2. Lottery ticket (20% weights): {lottery_ticket_loss:.6f} "
      f"({(full_test_loss/lottery_ticket_loss):.2f}x as good as full!)")
print(f"3. Random pruning: {random_pruned_loss:.6f} "
      f"({(random_pruned_loss/lottery_ticket_loss):.2f}x worse than lottery ticket)")
print(f"\n→ The 'winning ticket' subnetwork performs nearly as well as the full network!")
print(f"→ This explains why overparameterization helps: more chances to find a winning ticket.")

---

## Summary: Key Theoretical Insights

### 1. Universal Approximation
- ✓ Neural networks *can* represent any function (with enough neurons)
- ✓ Depth is more efficient than width for compositional functions
- ✓ Neural Tangent Kernel connects infinite-width networks to kernel methods
- ⚠ Representation ≠ Learnability: finding the weights is a separate challenge

### 2. Loss Landscape Geometry
- ✓ High-dimensional loss landscapes are easier to optimize than naive intuition suggests
- ✓ Most critical points are saddle points (escapable), not local minima (stuck)
- ✓ Different minima are connected by low-loss paths (mode connectivity)
- ✓ SGD naturally finds flat minima → better generalization

### 3. Implicit Regularization
- ✓ Gradient descent has implicit bias toward simpler solutions
- ✓ Double descent: overparameterization helps generalization (contrary to classical wisdom)
- ✓ Lottery Ticket Hypothesis: large networks contain trainable subnetworks
- ✓ Overparameterization helps *optimization*, not just *representation*

### Why This Matters for Practice

**Architecture design**:
- Prefer deeper networks over wider ones (when possible)
- Don't fear overparameterization
- Consider the inductive bias of your architecture

**Training**:
- Use SGD/mini-batch training (not full-batch)
- Early stopping provides implicit regularization
- Learning rate matters: affects which basin you reach

**Debugging**:
- If training fails, try a larger network (more lottery tickets)
- Check for sharp minima (high sensitivity to perturbations)
- Visualize loss landscapes to understand optimization difficulties

### Further Reading

**Universal Approximation**:
- Cybenko (1989): "Approximation by superpositions of a sigmoidal function"
- Hornik et al. (1989): "Multilayer feedforward networks are universal approximators"
- Jacot et al. (2018): "Neural Tangent Kernel: Convergence and Generalization in Neural Networks"

**Loss Landscapes**:
- Goodfellow et al. (2015): "Qualitatively characterizing neural network optimization problems"
- Garipov et al. (2018): "Loss Surfaces, Mode Connectivity, and Fast Ensembling of DNNs"
- Li et al. (2018): "Visualizing the Loss Landscape of Neural Nets"

**Implicit Regularization**:
- Zhang et al. (2017): "Understanding deep learning requires rethinking generalization"
- Nakkiran et al. (2020): "Deep Double Descent: Where Bigger Models and More Data Hurt"
- Frankle & Carbin (2019): "The Lottery Ticket Hypothesis: Finding Sparse, Trainable Neural Networks"

### Final Thoughts

These theoretical insights reveal that deep learning's success is not accidental:
- The geometry of high-dimensional spaces is actually helpful
- Optimization algorithms have beneficial implicit biases
- Overparameterization is a feature, not a bug

Understanding *why* neural networks work helps us:
- Design better architectures
- Choose appropriate training procedures
- Debug failures more effectively
- Push the boundaries of what's possible

The gap between theory and practice is closing, but many mysteries remain. Keep experimenting, keep learning!